In [27]:
%reload_ext autoreload
%autoreload 2
import os
import numpy as np
import time
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import tensorflow as tf
import pickle

In [58]:
from recommand import get_recommandation_result, get_k_accuracy
from utils import load_poi2region, get_sequences, digitize_datetime, group_data_by_id, \
    devide_train_and_test, get_ST_embeddings, get_emb_from_ckpt
from tqdm import tqdm
from dataloader import load_data, group_data_by_id
from parser import get_parser
from stsg_utils import save_emb_from_ckpt, load_model_tf
from model import STSkipgram
from stsg_parser import get_parser as model_parser

In [31]:
args = get_parser(['--CITY', 'NYC'])
args.pattern = 'hand'
args.use_decay = True

In [8]:
origin_data, dicts = load_data(os.path.join(args.ROOT, 'data','{}_INTV_processed_voc5_len2_setting_WITH_GPS_WITH_TIME_WITH_USERID.pk'.format(args.CITY) ))
poi2region = load_poi2region(os.path.join('data', '{}_poi2region.pk'.format(args.CITY)))
seqs = get_sequences(origin_data)
trainset, testset = devide_train_and_test(seqs)

Loading data from data/NYC_poi2region.pk


In [ ]:
sem_dim = 50; geo_dim = 100-sem_dim
logdir = os.path.join(args.ROOT, 'STSkipgram', 'benchmark')
model_args = model_parser(['--CITY', 'NYC', '--LOG_DIR', logdir, '--geo_reg_type', 'xn',\
                           '--sem_dim', str(sem_dim), '--geo_dim', str(geo_dim)])

In [ ]:
score = get_recommandation_result(testset, args, poi2region, get_emb_from_ckpt(model_args), mode='STSG')
acc = get_k_accuracy(score)
with open(os.path.join(model_args.LOG_DIR, 'accuracy.pk'), 'wb') as f:
    pickle.dump(acc, f)

---

In [10]:
sts, ste, stt = get_ST_embeddings(args)

Reading dadta from /home/haibin2/data/checkins/STSkipgram/benchmark/NYC_sem_emb.h5
Reading dadta from /home/haibin2/data/checkins/STSkipgram/benchmark/NYC_embeddings.h5
Reading dadta from /home/haibin2/data/checkins/STSkipgram/benchmark/NYC_time_embeddings.h5


In [23]:
r = multiprocess_recommand(args.nProcess, testset, args, poi2region, (sts, ste, stt), mode='STSG')

100%|██████████| 209/209 [00:32<00:00,  6.45it/s]


Job Done, used time 32.700907707214355
